In [216]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from category_encoders.one_hot import OneHotEncoder

## Setup e Importação

In [3]:
# Caminho do arquivo CSV no Google Drive
csv_file = os.getcwd() +"\\dados\\dados_tuberculose_2013_2023_limpo.csv"
xlsx_file = os.getcwd() +"\\dados\\pop_residente_municipios.xlsx"

df_raw = pd.read_csv(csv_file, encoding='latin1', low_memory=False)
df_munic = pd.read_excel(xlsx_file)

In [4]:
dummy_var = ('CS_SEXO')

## Juntando as Tabelas

In [5]:
df_munic_tidy = df_munic.melt(id_vars=['MUNICIPIO', 'ID_MUNIC', 'NOME_MUNIC'], var_name='ANO', value_name='POP_RESIDENTE_MUNIC')

In [6]:
df_merge = df_raw.merge(df_munic_tidy, left_on=['ID_MUNIC_A','NU_ANO'], right_on=['ID_MUNIC', 'ANO'], how='left')

In [7]:
#Check da quantidade de linhas após merge
df_raw.shape[0] - df_merge.shape[0]

0

## Seleção de Variáveis e Tratamento


In [ ]:
# Criar lista das variáveis selecionadas
variaveis_independentes = [ 'IDADE_ANOS', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'TRATAMENTO',
    'RAIOX_TORA', 'TESTE_TUBE', 'FORMA', 'AGRAVAIDS', 'AGRAVALCOO', 'AGRAVDIABE',
    'AGRAVDOENC', 'AGRAVOUTRA', 'AGRAVDROGA', 'AGRAVTABAC', 'BACILOSC_E', 'CULTURA_ES', 'CULTURA_OU', 'HIV', 'HISTOPATOL',
    'RIFAMPICIN', 'ISONIAZIDA', 'ETAMBUTOL', 'ESTREPTOMI', 'PIRAZINAMI',
    'ETIONAMIDA','NU_COMU_EX', 'DOENCA_TRA', 'TRATSUP_AT',  
    'TPUNINOT', 'POP_LIBER', 'POP_RUA', 'POP_SAUDE', 'POP_IMIG',
    'BENEF_GOV', 'TEST_MOLEC', 'TEST_SENSI', 'ANT_RETRO', 'TRANSF']

variaveis_local = ['SG_UF_AT','SG_UF_2','ID_MUNIC_A']
variaveis_acompanhamento = ['BACILOSC_1', 'BACILOSC_2', 'BACILOSC_3', 'BACILOSC_4', 'BACILOSC_5', 'BACILOSC_6', 'BAC_APOS_6', 'BACILOS_E2', 'BACILOSC_O', 'SITUA_9_M','SITUA_12_M']
#Estas são variáveis que buscamos em bases externas, representantes das características dos municípios
variaveis_exogenas = ['POP_RESIDENTE_MUNIC']

# Novo DataFrame com as variáveis independentes e alvo
df_modelagem = df_merge[['ABANDONO'] + variaveis_independentes +['ID_MUNIC_A']+['SG_UF']+variaveis_exogenas + ['NU_ANO']].copy()

In [44]:
df_modelagem.columns.sort_values()

Index(['ABANDONO', 'AGRAVAIDS', 'AGRAVALCOO', 'AGRAVDIABE', 'AGRAVDOENC',
       'AGRAVDROGA', 'AGRAVOUTRA', 'AGRAVTABAC', 'ANT_RETRO', 'BACILOSC_E',
       'BENEF_GOV', 'CS_ESCOL_N', 'CS_GESTANT', 'CS_RACA', 'CS_SEXO',
       'CULTURA_ES', 'CULTURA_OU', 'DOENCA_TRA', 'ESTREPTOMI', 'ETAMBUTOL',
       'ETIONAMIDA', 'FORMA', 'HISTOPATOL', 'HIV', 'IDADE_ANOS', 'ID_MUNIC_A',
       'ID_OCUPA_N', 'ISONIAZIDA', 'NU_ANO', 'NU_COMU_EX', 'PIRAZINAMI',
       'POP_IMIG', 'POP_LIBER', 'POP_RESIDENTE_MUNIC', 'POP_RUA', 'POP_SAUDE',
       'RAIOX_TORA', 'RIFAMPICIN', 'SG_UF', 'TESTE_TUBE', 'TEST_MOLEC',
       'TEST_SENSI', 'TPUNINOT', 'TRANSF', 'TRATAMENTO', 'TRATSUP_AT'],
      dtype='object')

v4: 'ID_OCUPA_N', 'TESTE_TUBE', 'CULTURA_OU', 'ETIONAMIDA', 'NU_CONTATO','NU_COMU_EX', 'DOENCA_TRA'

<P> 
Comentários: 
<p>

- Retiro campo HIV, senod que já tenho AGRAVAIDS?   
- Variavel ETIONAMIDA está correta? No dicionario está mal definida  
- Variável OUTRAS pode ser retirada? Qual a proporção de nao informados dela? Não estou utilizando a OUTRAS_DES 
    - OUTRAS foi retirado
- Variáveis 'NU_CONTATO','NU_COMU_EX' não seguem regra de negócio. Há registros de maior NU_COMU_EX (numero de contatos examinados) do que NU_CONTATO (numero de contatos existentes). QUal confiar? Alem disso, NU_COMU_EX tem bastante 9. Será que o 9 foi automático?
    - NU_CONTATO foi retirado
- ANT_RETRO não está descrita no dicionário e, se for ANTIRRETROVIRAL, existem mais números do que 0,1,2,9. Vale a pena deixar essa variável?

In [20]:
#Linhas rascunho na exploração

df_raw[['NU_CONTATO','NU_COMU_EX']][df_raw['NU_CONTATO'] - df_raw['NU_COMU_EX'] < 0]
df_raw['ANT_RETRO'].value_counts()/df_raw.shape[0]
df_raw['NU_COMU_EX'].value_counts().sort_values(ascending=False).head(10)

NU_COMU_EX
0.0    437952
1.0    112642
2.0    108887
9.0    105757
3.0     87714
4.0     58082
5.0     36631
6.0     21847
7.0     12790
8.0      9979
Name: count, dtype: int64

In [45]:
df_modelagem.isna().sum().values

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       2195,    0], dtype=int64)

In [48]:
print('CS_SEXO:',df_modelagem['CS_SEXO'].unique())
print('ETIONAMIDA:',df_modelagem['ETIONAMIDA'].unique())

#Checar o significado dessas classificações

CS_SEXO: ['M' 'F']
ETIONAMIDA: ['2' '0' '1' '9']


In [47]:
#Retirando linhas de categorias com poucas observações
#Possiveis erros ou má interpretações

df_modelagem = df_modelagem[df_modelagem['CS_SEXO'] != 'I']
df_modelagem = df_modelagem[df_modelagem['ETIONAMIDA'] != '/']


In [49]:
#Tratamento Dummies
gender_mapping = {"F":1, "M":0}
df_modelagem[dummy_var] = df_modelagem[dummy_var].map(gender_mapping) 

In [50]:
#Convertendo para numerico
convert_to_num = ['ETIONAMIDA', 'POP_RESIDENTE_MUNIC']

for col in convert_to_num:
    df_modelagem[col] = pd.to_numeric(df_modelagem[col])

## Feature Engineering

### ID Município e Informações Externas

IBGE classifica os municípios baseado na quantidade de habitantes e localização: https://www.ibge.gov.br/apps/rural_urbano/#/home  
Comecei com uma classificação simples baseada no IBGE, mas considerar os conglomerados urbanos pode fazer mais sentido.  
_Ideia: clusterizar as cidades. Complicação: clusterizar baseado em que? Como evitar vazamento de dados?_

Classificação atualmente adotada (mais simples):

- Pequeno Porte I: Até 20.000 habitantes
- Pequeno Porte II: De 20.001 a 50.000 habitantes
- Médio Porte: De 50.001 a 100.000 habitantes
- Grande Porte: De 100.001 a 900.000 habitantes
- Metrópole: Acima de 900.000 habitantes 

In [51]:
# Definindo as faixas e os rótulos baseados na classificação do IBGE
faixas = [0, 20000, 50000, 100000, 900000, float('inf')]
rotulos = ['Pequeno I', 'Pequeno II', 'Médio', 'Grande', 'Metrópole']

df_modelagem['porte_municipio'] = pd.cut(df_modelagem['POP_RESIDENTE_MUNIC'], bins=faixas, labels=rotulos, right=False)

In [52]:
# Distribuição dos indivíduos em cada grupo de município
df_modelagem['porte_municipio'].value_counts()

porte_municipio
Metrópole     399255
Grande        373777
Pequeno II     97796
Médio          84925
Pequeno I      67922
Name: count, dtype: int64

#### Variáveis criadas a partir do feature engineering

In [127]:
variaveis_feature_engineering = ['porte_municipio']

## Exploração 

In [20]:
df_modelagem['IDADE_ANOS'].value_counts().sort_index()

IDADE_ANOS
0.00      2081
0.01       109
0.02        78
0.03        39
0.04        62
          ... 
115.00       3
116.00       3
117.00       1
118.00       1
119.00       1
Name: count, Length: 166, dtype: int64

In [22]:
df_modelagem['ID_OCUPA_N'].value_counts()

ID_OCUPA_N
9.0     775824
0.0     176469
99.0     37229
71.0      7778
51.0      6916
62.0      4268
52.0      2315
78.0      1989
41.0      1229
72.0      1198
35.0      1025
14.0       984
76.0       723
42.0       713
32.0       667
63.0       640
23.0       586
91.0       586
25.0       582
61.0       570
84.0       515
22.0       497
77.0       426
31.0       360
26.0       342
21.0       180
24.0       149
33.0       132
75.0       127
86.0       119
2.0        101
37.0        96
82.0        80
73.0        58
11.0        55
64.0        52
34.0        51
95.0        51
81.0        40
39.0        28
3.0         23
30.0        23
12.0        22
1.0         18
74.0        13
20.0         8
83.0         8
13.0         5
Name: count, dtype: int64

## Filtro dos Anos

A ficha de notificação deo SINAN mudou para a v5 no período, incluindo e retirando variáveis.  
Analisando os shifts da base [_inserir os gráficos do outro notebook aqui_], percebemos anomalias ao passar do tempo  
A fim de desenvolver um modelo mais próximo da implementação no Sistema de Saúde, vamos filtrar para trabalhar com os valores em duas situações:
- a partir de 2018  
- pós período pandêmico (a partir de 2021)


In [242]:
df_modelagem_2018 = df_modelagem[df_modelagem['NU_ANO'] > 2018].copy()
df_modelagem_2021 = df_modelagem[df_modelagem['NU_ANO'] > 2021].copy()

In [154]:
df_modelagem_2021['TRANSF'].value_counts().sort_index()

TRANSF
0.0        23
1.0      7054
2.0      9394
3.0      1502
4.0        65
9.0    202630
Name: count, dtype: int64

In [243]:
#Estas variáveis foram retiradas na v5 da ficha de notificação
#AGRAVAIDS já está contemplado na variável HIV. Esta abarca mais informações que aquela

variaveis_retiradas = ['TESTE_TUBE', 'ID_OCUPA_N','CULTURA_OU', 'AGRAVAIDS','RIFAMPICIN','ISONIAZIDA', 
                        'ETAMBUTOL', 'ESTREPTOMI', 'PIRAZINAMI','ETIONAMIDA', 'DOENCA_TRA']

df_modelagem_2021.drop(columns=variaveis_retiradas, inplace=True)

In [235]:
variaveis_independentes_finais = list(df_modelagem_2021.columns[df_modelagem_2021.columns.isin(variaveis_independentes)])

## Variáveis Finais para Modelagem (após feature engineering)

In [236]:
variaveis_finais = variaveis_independentes_finais + variaveis_feature_engineering

In [244]:
df_modelagem_2021 = df_modelagem_2021[variaveis_finais + ['SG_UF']+['ABANDONO']].copy()

## Pré-processamento

In [ ]:
print("Colunas com 0 e 1:", [col for col in df_modelagem_2021.columns if set(df_modelagem_2021[col].dropna().unique()).issubset({0, 1})])
print("Colunas com 1 e 2 e 9:", [col for col in df_modelagem_2021.columns if set(df_modelagem_2021[col].dropna().unique()).issubset({1, 2, 9})])

Colunas com 0 e 1: ['ABANDONO', 'CS_SEXO']
Colunas com 1 e 2 e 9: ['AGRAVALCOO', 'AGRAVDIABE', 'AGRAVDOENC', 'AGRAVDROGA', 'AGRAVTABAC', 'POP_LIBER', 'POP_RUA']


In [213]:
dummy_var = ['CS_SEXO']
num_var = ['IDADE_ANOS']
cat_var = list(df_modelagem_2021.columns[~df_modelagem_2021.columns.isin(dummy_var+num_var)])

### One Hot Enconding

In [245]:
ohe = OneHotEncoder(cols=cat_var, drop_invariant=True)
df_modelagem_2021_ohe = ohe.fit_transform(df_modelagem_2021)

## Segmentação Regiões Brasil

| Nome UF | Código IBGE |
| --- | --- |
|Rondônia|11|
|Acre|12|
|Amazonas|13|
|Roraima|14|
|Pará|15|
|Amapá|16|
|Tocantins|17|
|Maranhão|21|
|Piauí|22|
|Ceará|23|
|Rio Grande do Norte|24|
|Paraíba|25|
|Pernambuco|26|
|Alagoas|27|
|Sergipe|28|
|Bahia|29|
|Minas Gerais|31|
|Espírito Santo|32|
|Rio de Janeiro|33|
|São Paulo|35|
|Paraná|41|
|Santa Catarina|42|
|Rio Grande do Sul|43|
|Mato Grosso do Sul|50|
|Mato Grosso|51|
|Goiás|52|
|Distrito Federal|53|

_modelando primeiro somente com os dados pós pandemicos_

In [251]:
sudeste = [31, 32, 33, 35]
norte = [11,12,13,14,15,16,17]
nordeste = [21,22,23,24,25,26,27,28,29]
centro_oeste = [50, 51, 52, 53]
sul = [41, 42, 43]

# Filtrar para o estado de São Paulo (SG_UF == 35)
df_sudeste = df_modelagem_2021_ohe[df_modelagem_2021_ohe['SG_UF'].isin(sudeste)].copy()
df_norte = df_modelagem_2021_ohe[df_modelagem_2021_ohe['SG_UF'].isin(norte)].copy()
df_nordeste = df_modelagem_2021_ohe[df_modelagem_2021_ohe['SG_UF'].isin(nordeste)].copy()
df_centro_oeste = df_modelagem_2021_ohe[df_modelagem_2021_ohe['SG_UF'].isin(centro_oeste)].copy()
df_sul =df_modelagem_2021_ohe[df_modelagem_2021_ohe['SG_UF'].isin(sul)].copy()

# Confirmar o tamanho do novo DataFrame
print(f"Quantidade de registros para Sudeste: {df_sudeste.shape[0]}")
print(f"Quantidade de registros para Norte: {df_norte.shape[0]}")
print(f"Quantidade de registros para Nordeste: {df_nordeste.shape[0]}")
print(f"Quantidade de registros para Centro Oeste: {df_centro_oeste.shape[0]}")
print(f"Quantidade de registros para Sul: {df_sul.shape[0]}")

for df_for in [df_sudeste, df_norte, df_nordeste,df_centro_oeste,df_sul]:
    df_for.drop(columns='SG_UF', inplace=True)

Quantidade de registros para Sudeste: 96898
Quantidade de registros para Norte: 27881
Quantidade de registros para Nordeste: 57995
Quantidade de registros para Centro Oeste: 11266
Quantidade de registros para Sul: 26628


In [252]:
#Proporção de desfecho
pd.DataFrame({'sudeste':df_sudeste['ABANDONO'].value_counts()/df_sudeste.shape[0],
             'norte':df_norte['ABANDONO'].value_counts()/df_norte.shape[0],
             'nordeste':df_nordeste['ABANDONO'].value_counts()/df_nordeste.shape[0],
             'centro_oeste':df_centro_oeste['ABANDONO'].value_counts()/df_centro_oeste.shape[0],
             'sul':df_sul['ABANDONO'].value_counts()/df_sul.shape[0],
})

,sudeste,norte,nordeste,centro_oeste,sul
ABANDONO,,,,,
0,0.825301,0.833614,0.875472,0.843511,0.8188
1,0.174699,0.166386,0.124528,0.156489,0.1812


In [253]:
df_sudeste

,IDADE_ANOS,CS_SEXO,CS_GESTANT_2,CS_GESTANT_3,CS_GESTANT_4,CS_GESTANT_5,CS_GESTANT_6,CS_GESTANT_7,CS_GESTANT_8,CS_RACA_1,...,TRANSF_4,TRANSF_5,TRANSF_6,porte_municipio_1,porte_municipio_2,porte_municipio_3,porte_municipio_4,porte_municipio_5,porte_municipio_6,ABANDONO
86209,75.0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
86210,22.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
86211,44.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
171406,39.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
256850,35.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025955,77.0,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1025957,27.0,0,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1025958,62.0,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1025959,68.0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [254]:
# 2. Separar variável dependente (y) e independentes (X)
X_sudeste = df_sudeste.drop(columns='ABANDONO')
X_norte = df_norte.drop(columns='ABANDONO')
X_nordeste = df_nordeste.drop(columns='ABANDONO')
X_centro_oeste = df_centro_oeste.drop(columns='ABANDONO')
X_sul = df_sul.drop(columns='ABANDONO')


y_sudeste = df_sudeste['ABANDONO']
y_norte = df_norte['ABANDONO']
y_nordeste = df_nordeste['ABANDONO']
y_centro_oeste = df_centro_oeste['ABANDONO']
y_sul = df_sul['ABANDONO']


#### Subamostragem <p>
Devido a grande quantidade de linhas, iremos subamostrar para rodar os testes

In [257]:
X_sample_sudeste, _, y_sample_sudeste, _ = train_test_split(X_sudeste, y_sudeste, train_size=0.1, stratify=y_sudeste, random_state=42)
X_sample_norte, _, y_sample_norte, _ = train_test_split(X_norte, y_norte, train_size=0.1, stratify=y_norte, random_state=42)
X_sample_nordeste, _, y_sample_nordeste, _ = train_test_split(X_nordeste, y_nordeste, train_size=0.1, stratify=y_nordeste, random_state=42)
X_sample_centro_oeste, _, y_sample_centro_oeste, _ = train_test_split(X_centro_oeste, y_centro_oeste, train_size=0.1, stratify=y_centro_oeste, random_state=42)
X_sample_sul, _, y_sample_sul, _ = train_test_split(X_sul, y_sul, train_size=0.1, stratify=y_sul, random_state=42)

print(f"Quantidade de registros para Sudeste: {X_sample_sudeste.shape[0]}")
print(f"Quantidade de registros para Norte: {X_sample_norte.shape[0]}")
print(f"Quantidade de registros para Nordeste: {X_sample_nordeste.shape[0]}")
print(f"Quantidade de registros para Centro Oeste: {X_sample_centro_oeste.shape[0]}")
print(f"Quantidade de registros para Sul: {X_sample_sul.shape[0]}")

Quantidade de registros para Sudeste: 9689
Quantidade de registros para Norte: 2788
Quantidade de registros para Nordeste: 5799
Quantidade de registros para Centro Oeste: 1126
Quantidade de registros para Sul: 2662


In [258]:
#salvando as amostras
os.makedirs("dados/amostras_ohe", exist_ok=True)

amostras = {
    "sudeste": {
        "X_sample": X_sample_sudeste,
        "y_sample": y_sample_sudeste
    },
    "norte": {
        "X_sample": X_sample_norte,
        "y_sample": y_sample_norte
    },
    "nordeste": {
        "X_sample": X_sample_nordeste,
        "y_sample": y_sample_nordeste
    },
    "centro_oeste": {
        "X_sample": X_sample_centro_oeste,
        "y_sample": y_sample_centro_oeste
    },
    "sul": {
        "X_sample": X_sample_sul,
        "y_sample": y_sample_sul
    }
}

# Loop para salvar todos os DataFrames
for regiao, conjuntos in amostras.items():
    for tipo, df in conjuntos.items():
        nome_arquivo = f"dados/amostras_ohe/{tipo}_{regiao}.csv"
        df.to_csv(nome_arquivo, index=False)

### Separando Treino e Teste

In [259]:
X_train_sudeste, X_test_sudeste, y_train_sudeste, y_test_sudeste = train_test_split(X_sample_sudeste, y_sample_sudeste, test_size=0.3, random_state=42, stratify=y_sample_sudeste)
X_train_norte, X_test_norte, y_train_norte, y_test_norte = train_test_split(X_sample_norte, y_sample_norte, test_size=0.3, random_state=42, stratify=y_sample_norte)
X_train_nordeste, X_test_nordeste, y_train_nordeste, y_test_nordeste = train_test_split(X_sample_nordeste, y_sample_nordeste, test_size=0.3, random_state=42, stratify=y_sample_nordeste)
X_train_centro_oeste, X_test_centro_oeste, y_train_centro_oeste, y_test_centro_oeste = train_test_split(X_sample_centro_oeste, y_sample_centro_oeste, test_size=0.3, random_state=42, stratify=y_sample_centro_oeste)
X_train_sul, X_test_sul, y_train_sul, y_test_sul = train_test_split(X_sample_sul, y_sample_sul, test_size=0.3, random_state=42, stratify=y_sample_sul)


In [260]:
# Garante que a pasta 'train_test' exista
os.makedirs("dados/train_test_ohe", exist_ok=True)

#salvando os csv
dados = {
    "sudeste": {
        "X_train": X_train_sudeste,
        "X_test": X_test_sudeste,
        "y_train": y_train_sudeste,
        "y_test": y_test_sudeste
    },
    "norte": {
        "X_train": X_train_norte,
        "X_test": X_test_norte,
        "y_train": y_train_norte,
        "y_test": y_test_norte
    },
    "nordeste": {
        "X_train": X_train_nordeste,
        "X_test": X_test_nordeste,
        "y_train": y_train_nordeste,
        "y_test": y_test_nordeste
    },
    "centro_oeste": {
        "X_train": X_train_centro_oeste,
        "X_test": X_test_centro_oeste,
        "y_train": y_train_centro_oeste,
        "y_test": y_test_centro_oeste
    },
    "sul": {
        "X_train": X_train_sul,
        "X_test": X_test_sul,
        "y_train": y_train_sul,
        "y_test": y_test_sul
    }
}

# Loop para salvar todos os DataFrames
for regiao, conjuntos in dados.items():
    for tipo, df in conjuntos.items():
        nome_arquivo = f"dados/train_test_ohe/{tipo}_{regiao}.csv"
        df.to_csv(nome_arquivo, index=False)